In [1]:
### Needed for google colab runtime
# import sys
# !{sys.executable} -m pip install yfinance yahoo_fin --upgrade --no-cache-dir

# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
import os
from pandas_datareader import data as pdr
from yahoo_fin import stock_info as si
import asyncio
from pandas import ExcelWriter
import yfinance as yf
import pandas as pd
import numpy as np
import datetime
import time
yf.pdr_override()

In [3]:
### Variables
tickers = pd.DataFrame()
tickers = pd.read_excel("https://topforeignstocks.com/wp-content/uploads/2022/07/Complete-List-of-SP-500-Constituents-July-4-2022.xlsx",sheet_name=0,header=None,usecols = "C,D",skiprows = 1,names = ["ticker","sector"])
tickers["ticker"] = [item.replace(".", "-") for item in tickers["ticker"]] # Yahoo Finance uses dashes instead of dots
tickers = tickers.set_index("sector")

#combine into one string needed to download multiple files using yfinance module
tickers_string = ""
for i in tickers["ticker"]:
    tickers_string += i + " "

index_name = '^SPX' # SP 500
start_date = datetime.datetime.now() - datetime.timedelta(days=365)
end_date = datetime.date.today()

In [4]:
###  Download ticker data from yahoo finance simultaneously
results = yf.download(tickers = tickers_string,start = start_date, end = end_date, interval= '1d',threads = True,group_by="ticker")

### write to file
for ticker in tickers["ticker"]:
    df = results[ticker].reset_index()
    df.to_csv( os.path.join(os.getcwd(),'us_stocks',ticker+'.csv'), index = False, header=True )

[*********************100%***********************]  505 of 505 completed

1 Failed download:
- CASH_USD: No data found, symbol may be delisted


In [5]:
# Index Returns
index_df = pdr.get_data_yahoo(index_name, start_date, end_date)
index_return = index_df['Adj Close'].iloc[-1] / index_df['Adj Close'].iloc[0]       # not efficient: (index_df['Percent Change'] + 1).cumprod()[-1]

# Import historical data as CSV for all stocks (makes the process faster)
df_list = pd.concat(( map(pd.read_csv, [os.path.join(os.getcwd(),'us_stocks',s+'.csv') for s in tickers["ticker"] ])), keys=tickers["ticker"] )

# Find top 30% performing stocks (relative to the index)
returns_multiples = []
for ticker in tickers["ticker"]:
    # Calculating returns relative to the market (returns multiple)
    df = df_list.loc[ticker].copy()
    stock_return = df['Adj Close'].iloc[-1] / df['Adj Close'].iloc[0]   # not efficient: ((df['Percent Change'] + 1).cumprod()).iloc[-1]
    
    returns_multiple = round((stock_return / index_return), 2)
    returns_multiples.extend([returns_multiple])
    
    print (f'Ticker: {ticker}; Returns Multiple against index: {returns_multiple}\n')

[*********************100%***********************]  1 of 1 completed
Ticker: AAPL; Returns Multiple against index: 1.06

Ticker: MSFT; Returns Multiple against index: 0.9

Ticker: AMZN; Returns Multiple against index: 0.77

Ticker: GOOGL; Returns Multiple against index: 0.79

Ticker: GOOG; Returns Multiple against index: 0.79

Ticker: TSLA; Returns Multiple against index: 1.22

Ticker: BRK-B; Returns Multiple against index: 1.03

Ticker: UNH; Returns Multiple against index: 1.32

Ticker: JNJ; Returns Multiple against index: 1.02

Ticker: XOM; Returns Multiple against index: 1.87

Ticker: META; Returns Multiple against index: 0.45

Ticker: NVDA; Returns Multiple against index: 0.64

Ticker: PG; Returns Multiple against index: 0.99

Ticker: JPM; Returns Multiple against index: 0.78

Ticker: V; Returns Multiple against index: 0.92

Ticker: PFE; Returns Multiple against index: 1.08

Ticker: HD; Returns Multiple against index: 0.92

Ticker: CVX; Returns Multiple against index: 1.73

Ticker:

In [6]:
# Creating dataframe of only top 40%
tickers["Returns_multiple"] =  returns_multiples
tickers["RS_Rating"] = tickers.groupby("sector").Returns_multiple.rank(pct=True) * 100
rs_df = tickers[tickers.RS_Rating >= tickers.RS_Rating.quantile(.60)].sort_values('RS_Rating',ascending=False)

In [7]:
# Checking Minervini conditions of top 40% of stocks in given list
exportList = pd.DataFrame(columns=['Stock', "Sector", "RS_Rating", "returns multiple", "Last Price","%change","50 Day MA", "150 Day Ma", "200 Day MA", "52 Week Low", "52 week High", "% from 52 week high"])

for i in range(len(rs_df)):
    stock = rs_df["ticker"].iloc[i]
    ret_mult = rs_df["Returns_multiple"].iloc[i]
    sector = rs_df.index[i]
    try:
        df = pd.read_csv(os.path.join(os.getcwd(),'us_stocks',f'{stock}.csv'), index_col=0) 
        sma = [50, 150, 200]
        for x in sma:
            df["SMA_"+str(x)] = round(df['Adj Close'].rolling(window=x).mean(), 2)
        
        # Storing required values 
        currentClose = df["Adj Close"][-1]
        prevClose =  df["Adj Close"][-2]
        pct_chg   = (currentClose - prevClose ) * 100/ prevClose
        moving_average_50 = df["SMA_50"][-1]
        moving_average_150 = df["SMA_150"][-1]
        moving_average_200 = df["SMA_200"][-1]
        low_of_52week = round(min(df["Low"]), 2)
        high_of_52week = round(max(df["High"]), 2)
        pct_from_52wkhigh = (high_of_52week - currentClose)/high_of_52week * 100
        RS_Rating = round(rs_df[rs_df['ticker']==stock].RS_Rating.tolist()[0])
        
        try:
            moving_average_200_20 = df["SMA_200"][-20]
        except Exception:
            moving_average_200_20 = 0

        # Condition 1: Current Price > 150 SMA and > 200 SMA
        condition_1 = currentClose > moving_average_150 > moving_average_200
        
        # Condition 2: 150 SMA and > 200 SMA
        condition_2 = moving_average_150 > moving_average_200

        # Condition 3: 200 SMA trending up for at least 1 month
        condition_3 = moving_average_200 > moving_average_200_20
        
        # Condition 4: 50 SMA> 150 SMA and 50 SMA> 200 SMA
        condition_4 = moving_average_50 > moving_average_150 > moving_average_200
           
        # Condition 5: Current Price > 50 SMA
        condition_5 = currentClose > moving_average_50
           
        # Condition 6: Current Price is at least 30% above 52 week low
        condition_6 = currentClose >= (1.3*low_of_52week)
           
        # Condition 7: Current Price is within 30% of 52 week high
        condition_7 = currentClose >= (.7*high_of_52week)

        # Condition 8: Daily change is positive 
        #
        
        # If all conditions above are true, add stock to exportList
        if(condition_1 and condition_2 and condition_3 and condition_4 and condition_5 and condition_6 and condition_7):
            exportEntry = pd.DataFrame({'Stock': stock, "Sector" : sector, "RS_Rating": RS_Rating , "returns multiple" : ret_mult, "Last Price" : round(currentClose,3), "%change" : round(pct_chg,2), "50 Day MA": moving_average_50, "150 Day Ma" : moving_average_150, "200 Day MA": moving_average_200, "52 Week Low": low_of_52week, "52 week High": high_of_52week, "% from 52 week high" : round(pct_from_52wkhigh,2)},index=[0])
            exportList = pd.concat([exportList,exportEntry], ignore_index=True)
            print (stock + " made the Minervini requirements")
    except Exception as e:
        print (e)
        print(f"Could not gather data on {stock}")

exportList = exportList.sort_values(by='RS_Rating', ascending=False)
print('\n', exportList)
writer = ExcelWriter(os.path.join(os.getcwd(),"output_screener_us_long_SP500.xlsx"))
exportList.to_excel(writer, "Sheet1",index=False)
writer.save()

ENPH made the Minervini requirements
NLSN made the Minervini requirements
OXY made the Minervini requirements
TMUS made the Minervini requirements
ED made the Minervini requirements
IRM made the Minervini requirements
MCK made the Minervini requirements
CF made the Minervini requirements
NOC made the Minervini requirements
MTB made the Minervini requirements
VRTX made the Minervini requirements
AZO made the Minervini requirements
GWW made the Minervini requirements
PGR made the Minervini requirements
CNC made the Minervini requirements
GPC made the Minervini requirements
SRE made the Minervini requirements
CHRW made the Minervini requirements
AJG made the Minervini requirements
DVN made the Minervini requirements
CI made the Minervini requirements
JKHY made the Minervini requirements
PWR made the Minervini requirements
LW made the Minervini requirements
ELV made the Minervini requirements
CTVA made the Minervini requirements
HII made the Minervini requirements
MOH made the Minervini re

In [8]:
exportList

,Stock,Sector,RS_Rating,returns multiple,Last Price,%change,50 Day MA,150 Day Ma,200 Day MA,52 Week Low,52 week High,% from 52 week high
0,ENPH,Information Technology,100,1.93,305.70,-3.69,259.93,203.33,197.34,113.40,324.84,5.89
5,IRM,Real Estate,100,1.24,55.23,4.60,50.67,50.50,49.45,41.67,58.61,5.77
7,CF,Materials,100,2.31,103.68,1.19,96.67,94.15,87.11,44.83,119.60,13.31
6,MCK,Health Care,100,1.82,366.21,-1.07,346.72,318.77,299.08,193.89,375.23,2.40
1,NLSN,Industrials,100,1.42,27.86,-0.04,25.46,24.03,23.01,16.02,27.92,0.21
4,ED,Utilities,100,1.40,100.85,0.19,96.35,92.79,89.83,71.52,101.57,0.71
3,TMUS,Communication Services,100,1.11,145.70,0.23,141.25,132.59,127.66,101.51,148.04,1.58
2,OXY,Energy,100,2.64,65.61,1.75,64.13,59.08,52.31,24.94,77.13,14.94
8,NOC,Industrials,99,1.41,491.41,0.34,473.51,452.93,433.71,344.89,497.20,1.16
9,MTB,Financials,98,1.42,187.61,-0.15,173.98,171.42,167.99,131.42,193.42,3.00
